In [27]:
def make_graph(n_vertices):
    return [[] for _ in range(n_vertices+1)]
    
def add_edge(graph, source, target):
    if target not in graph[source]:
        graph[source].append(target)
        
g = make_graph(5)
print(g)

add_edge(g, 0, 1)
add_edge(g, 0, 5)
add_edge(g, 1, 2)
add_edge(g, 1, 3)
add_edge(g, 2, 4)
add_edge(g, 3, 5)
add_edge(g, 5, 1)


print(g)

[[], [], [], [], [], []]
[[1, 5], [2, 3], [4], [5], [], [1]]


In [35]:
def dfs(graph, v, t, current_path, on_path, shortest_ref):
    if v == t:
        shortest = shortest_ref[0]
        if shortest is None or len(current_path) < len(shortest):
            shortest_ref[0] = current_path[:]
    else:
        for w in graph[v]:
            if not on_path[w]:
                current_path.append(w)
                on_path[w] = True
                dfs(graph, w, t, current_path, on_path, shortest_ref)
                current_path.pop()
                on_path[w] = False
    

def dfs_path(graph, s, t):
    shortest_ref = [None]
    on_path = [False] * len(graph)
    current_path = [s]
    on_path[s] = True
    dfs(graph, s, t, current_path, on_path, shortest_ref)
    return shortest_ref[0]

print(dfs_path(g, 0, 4))
print(dfs_path(g, 0, 5))

[0, 1, 2, 4]
[0, 5]


In [23]:
def argmin_seen(dist, seen, processed):
    v, d = None, float("Inf")
    for i, dd in enumerate(dist):
        if seen[i] and not processed[i] and dd < d:
            v, d = i, dd
    return v

def dijkstra(graph, s):
    seen = [False] * len(graph)
    processed = [False] * len(graph)
    dist = [float("Inf")] * len(graph)
    
    dist[s] = 0
    seen[s] = True
    v = argmin_seen(dist, seen, processed)
    while v is not None:
        for w in graph[v]:
            if not processed[w]:
                dist[w] = min(dist[w], dist[v] + 1)
                seen[w] = True
        processed[v] = True
        v = argmin_seen(dist, seen, processed)
    
    return dist
    
dist = dijkstra(g, 0)
print(dist)

[0, 1, 2, 2, 3, 1]


In [26]:
def reverse_graph(graph):
    reverse = make_graph(len(graph)-1)
    for v in range(len(graph)):
        for w in graph[v]:
            add_edge(reverse, w, v)
    return reverse

def back_track(graph, dist, s, t):
    reverse = reverse_graph(graph)
    reverse_path = [t]
    v = t
    while v != s:
        for w in reverse[v]:
            if dist[v] == dist[w] + 1:
                reverse_path.append(w)
                v = w
                break
    return list(reversed(reverse_path))

print(reverse_graph(g))
print(back_track(g, dist, 0, 4))
print(back_track(g, dist, 0, 5))

[[], [0, 5], [1], [1], [2], [0, 3]]
[0, 1, 2, 4]
[0, 5]
